<a href="https://colab.research.google.com/github/minsaee/ai_chat_openAI/blob/master/006_LangchainChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/ai_chat_openAI'

Mounted at /content/drive
/content/drive/MyDrive/ai_chat_openAI


In [ ]:
!pip install langchain openai==0.28.1 tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.3/205.3 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 여러 문서에서 찾아서 답변하는 챗봇 만들기

랭체인(langchain)이란?  
 - ChatGPT와 같은 대규모 언어 모델을 사용하여 어플리케이션을 쉽게 생성할 수 있도록 설계된 프레임워크이다.  
 - LLM에 외부의 '지식'이나 '계산능력'을 활용할 때 도움이 된다.  
 - 자신이 학습한 것만으로 대화하는 LLM에게 '책'이나 '프로그램'을 전달해서 외부의 '지식'이나 '계산능력'을 활용할 수 있게 하는 역할을 한다.

In [ ]:
import os

os.environ["OPENAI_API_KEY"]='sk-'

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

### Load multiple and process documents

In [ ]:
# !wget https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip
# !unzip data

In [ ]:
# path : directory
# glob : file

loader = DirectoryLoader('./data', glob='*.txt', loader_cls=TextLoader)

documents = loader.load()

print('문서의 개수 :', len(documents))

문서의 개수 : 57


In [ ]:
print('1번 문서 :', documents[1])
print('-'*20)
print('21번 문서:', documents[2])
print('-'*20)

1번 문서 : page_content='정책제목: 청년 매입임대주택 공급\n서울시청 주택정책과에서는 생활이 어려운 저소득층, 저소득 청년층, 그리고 신혼부부를 위해 주거안정을 지원하기 위한 정책을 운영하고 있습니다. 이 정책은 중앙정부, 서울시, 그리고 서울주택도시공사의 재정과 주택도시기금을 활용하여 서울주택도시공사가 주택을 매입하여 주변시세보다 저렴한 가격으로 공급하는 것을 목표로 합니다.\n\n이 정책은 2023년 1월 31일부터 2023년 12월 31일까지 운영되며, 사업 신청 기간은 2023년 6월 1일부터 2023년 12월 31일까지입니다. 이 정책은 총 1,000호의 주택을 공급하며, 임대 기간은 계약 기간 2년이며, 계약 연장은 2회까지 가능합니다. 결혼 시 최대 20년까지 거주할 수 있으며, 입주자격을 유지하는 경우 2회 한해 재계약이 가능합니다. 추가적으로 2회 재계약 이후에도 연장을 원할 경우 해당 세대의 월평균 소득과 자산 기준을 충족해야 하며, 최장 20년까지 재계약이 가능합니다.\n\n이 정책의 임대 조건은 시중 전가 시세(감정평가액)의 30% 수준으로 공급되며, 임대 보증금은 100만원입니다. 신청자격은 19세에서 39세까지의 저소득층과 저소득 청년층을 대상으로 하며, 주택 공급 신청자는 무주택이어야 하고 도시근로자의 월평균 소득이 100% 이하이며 자산 기준을 충족해야 합니다.\n\n주택 신청은 서울주택도시공사의 인터넷 홈페이지(www.i-sh.co.kr)를 통해 접수되며, 해당 사이트에서 신청 방법과 신청서류에 대한 자세한 정보를 확인하실 수 있습니다. 신청자격 확인, 인적 사항 작성, 가점 사항 입력 등의 절차를 거치게 됩니다.\n\n이 정책은 서울주택도시공사에서에서 주관하며, 상세한 심사 및 발표 일정은 입주자 모집공고를 통해 안내됩니다. 입주자 모집 공고일, 주택 공개 기간, 인터넷 신청 접수 기간, 서류 제출 기간, 당첨자 발표일, 계약 체결일, 입주일 등이 정해져 있습니다.\n\n신청자들은 주택도시공사의 인터넷 홈페이지에서

In [ ]:
documents[1]

Document(page_content='정책제목: 청년 매입임대주택 공급\n서울시청 주택정책과에서는 생활이 어려운 저소득층, 저소득 청년층, 그리고 신혼부부를 위해 주거안정을 지원하기 위한 정책을 운영하고 있습니다. 이 정책은 중앙정부, 서울시, 그리고 서울주택도시공사의 재정과 주택도시기금을 활용하여 서울주택도시공사가 주택을 매입하여 주변시세보다 저렴한 가격으로 공급하는 것을 목표로 합니다.\n\n이 정책은 2023년 1월 31일부터 2023년 12월 31일까지 운영되며, 사업 신청 기간은 2023년 6월 1일부터 2023년 12월 31일까지입니다. 이 정책은 총 1,000호의 주택을 공급하며, 임대 기간은 계약 기간 2년이며, 계약 연장은 2회까지 가능합니다. 결혼 시 최대 20년까지 거주할 수 있으며, 입주자격을 유지하는 경우 2회 한해 재계약이 가능합니다. 추가적으로 2회 재계약 이후에도 연장을 원할 경우 해당 세대의 월평균 소득과 자산 기준을 충족해야 하며, 최장 20년까지 재계약이 가능합니다.\n\n이 정책의 임대 조건은 시중 전가 시세(감정평가액)의 30% 수준으로 공급되며, 임대 보증금은 100만원입니다. 신청자격은 19세에서 39세까지의 저소득층과 저소득 청년층을 대상으로 하며, 주택 공급 신청자는 무주택이어야 하고 도시근로자의 월평균 소득이 100% 이하이며 자산 기준을 충족해야 합니다.\n\n주택 신청은 서울주택도시공사의 인터넷 홈페이지(www.i-sh.co.kr)를 통해 접수되며, 해당 사이트에서 신청 방법과 신청서류에 대한 자세한 정보를 확인하실 수 있습니다. 신청자격 확인, 인적 사항 작성, 가점 사항 입력 등의 절차를 거치게 됩니다.\n\n이 정책은 서울주택도시공사에서에서 주관하며, 상세한 심사 및 발표 일정은 입주자 모집공고를 통해 안내됩니다. 입주자 모집 공고일, 주택 공개 기간, 인터넷 신청 접수 기간, 서류 제출 기간, 당첨자 발표일, 계약 체결일, 입주일 등이 정해져 있습니다.\n\n신청자들은 주택도시공사의 인터넷 홈페이지에

### Split texts(텍스트 분할하기)

- ChatGPT API의 GPT-3.5-turbo 모델은 입력으로 사용하는 텍스트와 ChatGPT API가 반환하는 답변의 길이를 모두 포함하여 최대 4,096 토큰을 처리할 수 있다.
- 예를 들어, ChatGPT API 입력으로 약 3,000토큰 정도의 길이를 가진 입력을 사용하면 ChatGPT는 최대 약 1,000토큰의 답변을 할 수 있는 구조이다.
- 입력의 길이가 4,096의 길이에 인접한 입력을 사용하면 ChatGPT API가 답변을 하던 도중 4,096개의 토큰을 모두 소진하면 더 이상 답변을 생성할 수 없어 텍스트가 중간에 끊길 수 있다. 또한 입력의 길이가 4,096을 넘으면 ChatGPT의 입력으로 사용할 수 없다는 에러가 발생한다.
- 이런 현상 때문에 너무 긴 문서들은 한 번에 입력으로 사용할 수 없으므로 여러 개의 뭉치(chunk)로 자른 후 ChatGPT API를 여러 번 호출하여 처리해야 한다.
- 랭체인의 RecursiveCharacterTextSplitter는 이를 위해 사용하는 도구로, 주어진 문서를 더 작은 단위의 텍스트 뭉치(chunk)로 잘라주는 역할을 한다.
-chunk_size에는 텍스트의 분할 기준이 되는 길이를 설정하고, chunk_overlap에는 텍스트를 분할할 때 텍스트의 내용을 얼마나 겹쳐서 자를 것인지 지정한다.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

print('분할된 텍스트의 개수:', len(texts))

분할된 텍스트의 개수: 64


In [ ]:
source_list = []
for i in range(0, len(texts)):
  source_list.append(texts[i].metadata['source'])

element_counts = Counter(source_list)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print('2개 이상으로 분할된 문서:', filtered_counts)
print('분할된 텍스트 개수:', len(documents)+len(filtered_counts))

2개 이상으로 분할된 문서: {'data/31.txt': 2, 'data/48.txt': 2, 'data/40.txt': 2, 'data/36.txt': 2, 'data/49.txt': 2, 'data/22.txt': 2, 'data/23.txt': 2}
분할된 텍스트 개수: 64


### Create Chroma DB

- ChromaDB는 임베딩과 코사인 유사도 계산, 텍스트 불러오는 과정들을 기능별로 구현하여 사용자가 벡터를 좀 더 쉽게 다룰 수 있게 도와주는 벡터응용 도구이다.
- Chroma.from_documents()함수를 이용해 벡터 도구 객체를 선언한다. 이때 documents에는 벡터화의 단위가 될 텍스트 리스트를 매개변수로 전달하고, embedding에는 어떤 종류의 임베딩을 사용할 것인지를 기재한다.

In [ ]:
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embedding
)

vectordb

### Make a retriever
 - 벡터 도구 객체를 선언하고 나면 as_retriever()함수를 이용해 입력된 텍스트로부터 유사한 텍스트를 찾아주는 검색기를 선언한다.

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents('신혼 부부를 위한 정책이 있어?')
print('유사 문서 개수:', len(docs))
print('--'*20)
print('첫번째 유사 문서:', docs[0])
print('--'*20)
print('각 유사 문서의 문서 출처:')
for doc in docs:
  print(doc.metadata["source"])

유사 문서 개수: 4
----------------------------------------
첫번째 유사 문서: page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.' metadata={'source': 'data/23.txt'}
----------------------------------------
각 유사 문서의 문서 출처:
data/23.txt
data/23.txt
data/23.txt
data/23.txt


### 결과를 k개 반환

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={'k':2})

In [ ]:
docs = retriever.get_relevant_documents('신혼 부부를 위한 정책이 있어?')
for doc in docs:
  print(doc.metadata['source'])
print(docs)

data/23.txt
data/23.txt
[Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.', metadata={'source': 'data/23.txt'}), Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6

chain_type = 'stuff'의 내부처리

```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

{텍스트}에는 입력데이터와 유사도가 높은 텍스트의 본문이 삽입되고,  
{질문}에는 입력 텍스트가 삽입되는 구조이다.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0),
    chain_type = 'stuff',
    retriever = retriever,
    return_source_documents = True)

### Query

In [ ]:
input_text = '대출과 관련된 정책이 궁금합니다.'
chatbot_response = qa_chain(input_text)
print(chatbot_response)

{'query': '대출과 관련된 정책이 궁금합니다.', 'result': '서울시 학자금대출 신용회복 지원사업은 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 지원 정책입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공하여 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. \n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n자세한 내용은 신용회복 신청 안내 사이트(https://youth.seoul.go.kr/site/main/board/notice/27789?baCategory1=basic&baCommSelec=true)를 참고하시기 바랍니다.', 'source_documents': [Document(page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩니다.\n\n신청은 서울청년포털(youth.seoul.go.kr)을 통해 신청할 수 있으며, 심사 및 발표는 매월 1~2회 선정되며, 신청인원에 따라 주기가 변동할 수 있습니다.\n\n이 정책은 서울시 미래청

In [ ]:
def get_chatbot_response(chatbot_response):
  print(chatbot_response['result'].strip())
  print('\n문서 출처:')
  for source in chatbot_response['source_documents']:
    print(source.metadata['source'])

In [ ]:
input_text = '신혼부부 정책 알려줘.'
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

신혼부부 정책은 서울시에서 운영하는 정책으로, 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 지원되며, 대상 가구는 총 8,000가구로 제한됩니다.

이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.

이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있으며, 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 필요합니다. 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.

문서 출처:
data/23.txt
data/23.txt


In [ ]:
chatbot_response

{'query': '신혼부부 정책 알려줘.',
 'result': '신혼부부 정책은 서울시에서 운영하는 정책으로, 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 마련되었습니다. 이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 지원되며, 대상 가구는 총 8,000가구로 제한됩니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있으며, 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 필요합니다. 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.',
 'source_documents': [Document(page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은

In [ ]:
input_text = '전세자금이 필요한 사람을 위한 정책 이름 찾아줘.'
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

해당 정책의 이름은 "희망두배 청년통장"입니다.

문서 출처:
data/40.txt
data/40.txt


In [ ]:
query = '희망두배 청년통장의 지원 조건은?'
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

희망두배 청년통장의 지원 조건은 다음과 같습니다:

1. 신청자격: 18세에서 34세까지의 저소득 근로 청년이며, 서울시 거주자이고 공고일 현재 근로 중인 자격 요건을 모두 충족하는 경우 신청할 수 있습니다.

2. 저축 목적: 월 10만원 또는 15만원의 저축액을 선택할 수 있으며, 저축기간은 2년 또는 3년입니다.

3. 지원 금액: 본인이 적립한 금액의 100%로 지원되며, 10만원 저축 시 2년형은 480만원 이상(이자 포함), 3년형은 720만원 이상(이자 포함), 15만원 저축 시 2년형은 720만원 이상(이자 포함), 3년형은 1,080만원 이상(이자 포함)의 지원이 이루어집니다.

신청은 방문접수, 우편 신청(도착분 기준 18:00까지 동주민센터 도착), 이메일 신청(접수 마감일 18:00까지 도착) 방법으로 이루어집니다. 신청 기한 내에 제출서류가 누락되거나 식별이 불가능한 경우 대상자 선정에서 제외될 수 있습니다. 이메일로 제출할 경우 원본 스캔 또는 이미지 촬영하여 PDF, JPG, PNG 파일 형식으로 제출해야 합니다.

자세한 신청 방법과 필요한 서류는 서울시복지재단, 서울시 및 25개 자치구 홈페이지에서 확인하실 수 있습니다.

문서 출처:
data/40.txt
data/40.txt


In [ ]:
query = '희망두배 청년통장의 마감 기한 알려줘.'
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

마감 기한은 문맥에서 명시되어 있지 않습니다. 따라서, 희망두배 청년통장의 마감 기한에 대해서는 알 수 없습니다. 자세한 신청 방법과 필요한 서류는 서울시복지재단의 홈페이지 및 서울시 안심돌봄복지과의 홈페이지에서 확인하실 수 있습니다.

문서 출처:
data/40.txt
data/40.txt


### Gradio로 챗봇 UI만들기

In [ ]:
!pip install gradio==3.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.4 MB/s eta 0:00

To create a public link, set

In [ ]:
import gradio as gr

# 인터페이스 생성
with gr.Blocks() as demo:
  chatbot = gr.Chatbot(label='청년정책챗봇') # 레이블 좌측 상단에 구성
  msg = gr.Textbox(label='질문해주세요!') # 레이블 하단에 구성
  clear = gr.Button('대화 초기화') # 초기화 버튼

  # 챗봇 답변 처리하는 함수
  def respond(message, chat_history):
    result = qa_chain(message)
    bot_message = result['result']
    bot_message += '# sources:'

    # 답변 출처 표기
    for i, doc in enumerate(result['source_documents']):
      bot_message += '['+str(i+1)+']' + doc.metadata['source'] + ' '

    # 채팅 기록에 사용자의 메세지와 봇의 응답을 추가
    chat_history.append((message, bot_message))
    return '', chat_history

  # 사용자의 입력을 제출하면 respond 함수가 호출
  msg.submit(respond, [msg, chatbot], [msg, chatbot])

  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
  clear.click(lambda: None, None, chatbot, queue = False)


demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
